# LinkedIn Queens Solver
## By Darci Peoples (darcipeoples.com)

### Puzzle Class and Solver

In [ ]:
from enum import Enum
from typing import List
from collections import defaultdict

import sys
sys.path.append("..")

from utils.utils import fmt

class Color(Enum):
    LIGHT_BLUE = 'L'
    ORANGE = 'O'
    BLUE = 'B'
    GREEN = 'G'
    MAGENTA = 'M'
    RED = 'R'
    YELLOW = 'Y'
    WHITE = 'W'
    BLACK = 'K'
    PURPLE = 'P'
    CYAN = 'C'

COLOR_ANSI = {
    Color.LIGHT_BLUE: 68,
    Color.ORANGE: 208,
    Color.BLUE: 27,
    Color.GREEN: 70,
    Color.MAGENTA: 13,
    Color.RED: 196,
    Color.YELLOW: 214,
    Color.WHITE: 244,
    Color.BLACK: 238,
    Color.PURPLE: 141,
    Color.CYAN: 45
}

class Marker(Enum):
    BLANK = ' '
    QUEEN = 'Q'
    NOT_QUEEN = '*'

class ContradictionException(Exception):
    pass

class IllegalMarkerChangeException(Exception):
    pass

class Cell:
    def __init__(self, i: int, j: int, color: Color, marker: Marker = Marker.BLANK):
        self.i = i
        self.j = j
        self.color = color
        self.marker = marker

class Board:
    def __init__(self, grid: List[List[Cell]]):
        self.grid = grid
        self.M = len(self.grid)
        self.N = len(self.grid[0])

    def parse_from_text(file_path: str):
        grid = [[Cell(i, j, Color(letter), Marker.BLANK) for j, letter in enumerate(line)] for i, line in enumerate(list(x.strip()) for x in open(file_path, 'r').readlines())]
        return Board(grid)

    def dump_text(self):
        output = [[fmt(f"{cell.color.value}", COLOR_ANSI[cell.color], True) for cell in row] for row in self.grid]
        return '\n'.join([''.join(line) for line in output]) + '\n'

    def __str__(self):
        output = [[fmt(f"{cell.marker.value} ", COLOR_ANSI[cell.color], True) for cell in row] for row in self.grid]
        return '\n'.join([' '.join(line) for line in output]) + '\n'
    
    # TODO: Maintain some maps instead of recalculating
    # Set marker, throw if it's an invalid change
    def _set_marker(self, i, j, marker):
        curr_marker = self.grid[i][j].marker

        # Ignore NOPs
        if curr_marker == marker:
            return
        
        # Don't allow clearing cells
        if marker == Marker.BLANK:
            raise IllegalMarkerChangeException()
        
        # Allow placing into a blank cell
        if curr_marker == Marker.BLANK:
            self.grid[i][j].marker = marker
            return
        
        # Don't allow Q <-> X (usually a bad sign)
        raise IllegalMarkerChangeException()

    def place_x(self, i, j):
        self._set_marker(i, j, Marker.NOT_QUEEN)
    
    # Set cell to a queen, throw if invalid, handle X'ing row, col, diagonals
    def place_queen(self, i, j):
        # Set the cell to Queen
        self._set_marker(i, j, Marker.QUEEN)

        # Set other cells in the column to X
        for ni in range(0, self.M):
            if ni != i:
                self.place_x(ni, j)

        # Set other cells in the row to X
        for nj in range(0, self.N):
            if nj != j:
                self.place_x(i, nj)

        # Set the 4 diagonal cells to X
        for di, dj in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
            ni, nj = i + di, j + dj
            if 0 <= ni < self.M and 0 <= nj < self.N:
                self.place_x(ni, nj)
        
        # Set other cells in the color region to X
        for ni in range(self.M):
            for nj in range(self.N):
                if self.grid[ni][nj].color == self.grid[i][j].color and (ni, nj) != (i, j):
                    self.place_x(ni, nj)

    # TODO: Make sure this is called repeatedly
    def fill_certain_queens_strategy(self):
        color_markers = defaultdict(lambda: defaultdict(list))
        row_markers = defaultdict(lambda: defaultdict(list))
        col_markers = defaultdict(lambda: defaultdict(list))

        # Find marker locations for each color, row, and col
        for i, row in enumerate(self.grid):
            for j, cell in enumerate(row):
                color_markers[cell.color][cell.marker].append((i, j))
                row_markers[i][cell.marker].append((i, j))
                col_markers[j][cell.marker].append((i, j))
        
        # If a color, row, or col has one blank, put a Q there
        for zone_freqs in [row_markers, col_markers, color_markers]:
            for freqs in zone_freqs.values():
                if len(freqs[Marker.BLANK]) == 1:
                    assert len(freqs[Marker.QUEEN]) == 0
                    bi, bj = freqs[Marker.BLANK][0]
                    self.place_queen(bi, bj)

    # If all blanks for k colors are on just k lines, k queens have to be there
    # So X out all other blanks in the lines
    def fill_multi_owned_line_strat(self):
        # For each color, find which rows and cols have blanks in them
        color_row_blanks = defaultdict(set)
        color_col_blanks = defaultdict(set)
        for i, row in enumerate(self.grid):
            for j, cell in enumerate(row):
                if cell.marker == Marker.BLANK:
                    color_row_blanks[cell.color].add(i)
                    color_col_blanks[cell.color].add(j)
        color_row_blanks = list(color_row_blanks.items())
        color_col_blanks = list(color_col_blanks.items())
        
        # If k colors span k lines, X out all other blanks in the lines
        def claim_multiple_lines(color_idxs, color_set, line_set, check_rows, max_lines):
            color_line_blanks = color_row_blanks if check_rows else color_col_blanks

            prev_color_idx = color_idxs[-1] if color_idxs else -1
            for next_color_idx in range(prev_color_idx + 1, len(color_line_blanks)):
                next_color, next_lines = color_line_blanks[next_color_idx]

                new_color_idxs = color_idxs + [next_color_idx]
                new_color_set = color_set | {next_color}
                new_lines = line_set | next_lines

                # If trying to merge all lines, give up
                if len(new_lines) == max_lines:
                    continue

                # If k colors span k lines, X out any other blanks in the lines
                if len(new_lines) == len(new_color_idxs):
                    for line in new_lines:
                        line_coords = [(line, j) for j in range(self.N)] if check_rows else [(i, line) for i in range(self.M)]
                        for i, j in line_coords:
                            if self.grid[i][j].color not in new_color_set:
                                self.place_x(i, j)
                                # TODO: Remove color from consideration in deeper levels?

                claim_multiple_lines(new_color_idxs, new_color_set, new_lines, check_rows, max_lines)
        
        claim_multiple_lines([], set(), set(), True, self.M)
        claim_multiple_lines([], set(), set(), False, self.N)

        # TODO: Make more efficient by only merging colors that have some range overlap
    
    # TODO: Strategy to x out border around 2 remaining blanks of a color.
    #              xx                    x
    # E.g. RR  ->  RR       or R R  ->  R R
    #              xx                    x
    # TODO: Also make work for like L shape that also can't have queen next to it

    # Check if a board is solved (both filled and valid)
    def is_solved(self):
        return self.is_filled() and self.is_valid()

    # Check that the board is all filled out
    def is_filled(self):
        for row in self.grid:
            for cell in row:
                if cell.marker == Marker.BLANK:
                    return False
        return True

    # Check that a board is valid
    def is_valid(self):
        color_markers = defaultdict(lambda: defaultdict(int))
        row_markers = defaultdict(lambda: defaultdict(int))
        col_markers = defaultdict(lambda: defaultdict(int))

        queen_cells = set()

        # Find marker locations for each color, row, and col
        # Also, find queen positions
        for i, row in enumerate(self.grid):
            for j, cell in enumerate(row):
                color_markers[cell.color][cell.marker] += 1
                row_markers[i][cell.marker] += 1
                col_markers[j][cell.marker] += 1
                
                if cell.marker == Marker.QUEEN:
                    queen_cells.add((i, j))
        
        # For each row, col, and color region
        # Ensure there is 1 queen or some blanks left
        for zone_markers in [row_markers, col_markers, color_markers]:
            for freqs in zone_markers.values():
                # If no queens and no blanks, invalid
                if freqs[Marker.QUEEN] == 0 and freqs[Marker.BLANK] == 0:
                    return False
                # If multiple queens, invalid
                if freqs[Marker.QUEEN] > 1:
                    return False
        
        # Check that queens aren't diagonal from each other
        for i, j in queen_cells:
            for di, dj in [(-1, -1), (-1, 1), (1, -1), (1, 1)]:
                ni, nj = i + di, j + dj
                if 0 <= ni < self.M and 0 <= nj < self.N and self.grid[ni][nj].marker == Marker.QUEEN:
                    return False
        
        return True

    def __eq__(self, other):
        if self.M != other.M:
            return False
        if self.N != other.N:
            return False
        for i in range(self.M):
            for j in range(self.N):
                cell = self.grid[i][j]
                other_cell = other.grid[i][j]
                if cell.color != other_cell.color:
                    return False
                if cell.marker != other_cell.marker:
                    return False
        return True

    def copy(self):
        grid = []
        for i in range(self.M):
            row = []
            for j in range(self.N):
                other_cell = self.grid[i][j]
                cell = Cell(i, j, other_cell.color, other_cell.marker)
                row.append(cell)
            grid.append(row)
        return Board(grid)

# Return a blank for us to guess on. It'll be the top-left one of the smallest zone.
def pick_a_blank(board):
    color_blanks = defaultdict(set)
    row_blanks = defaultdict(set)
    col_blanks = defaultdict(set)

    # Find blank locations for each color, row, and col
    for i, row in enumerate(board.grid):
        for j, cell in enumerate(row):
            if cell.marker == Marker.BLANK:
                color_blanks[cell.color].add((i, j))
                row_blanks[i].add((i, j))
                col_blanks[j].add((i, j))
    
    # Find the smallest zone
    best_zone_idxs = None
    for zone_blanks in [color_blanks, row_blanks, col_blanks]:
        for blank_idxs in zone_blanks.values():
            if not best_zone_idxs or len(blank_idxs) < len(best_zone_idxs):
                best_zone_idxs = blank_idxs
    
    # Pick the top left blank of the smallest zone
    return min(best_zone_idxs)

def solve(board):
    # Apply strategies until the board doesn't change
    old_board = None
    while not old_board or board != old_board:
        old_board = board.copy()
        board.fill_multi_owned_line_strat()
        board.fill_certain_queens_strategy()

    # Board is invalid
    if not board.is_valid():
        raise ContradictionException('Board is invalid')

    # Board is solved (valid and full)
    if board.is_filled():
        return board
    
    # Try placing a queen in a blank spot
    i, j = pick_a_blank(board)
    try:
        board.place_queen(i, j)
        solution = solve(board)
    # If that didn't end up working out, it can't be a queen
    except (ContradictionException, IllegalMarkerChangeException) as e:
        board = old_board
        board.place_x(i, j)
        solution = solve(board)

    return solution

### Image Parsing

In [2]:
# This cell has generic image parsing helpers

from PIL import Image, ImageDraw
import numpy as np

from utils.utils import find_boxes, draw_boxes

# Parse a screenshot from LinkedIn
def parse_linkedin_image(image_path, debug=False):
    # TODO: Make debug line width thicker for larger images

    # Parameters for find_boxes, for all colored cells
    # TODO: Rename
    target_colors = {
        # Color: box_color, min_size, remove_nested, max_size, tolerance, merge_overlapping
        '': ([[163, 210, 216], [255, 201, 146], [150, 190, 255], [179, 223, 160], [223, 160, 191], [255, 123, 96], 
              [230, 243, 136], [223, 223, 223], [185, 178, 158], [187, 163, 226], [98, 239, 234]], 
              'white', 90, False, 180, 2, False)
    }

    # Colors used to classify cells
    cell_colors = {
        Color.LIGHT_BLUE: [163, 210, 216],
        Color.ORANGE: [255, 201, 146],
        Color.BLUE: [150, 190, 255],
        Color.GREEN: [179, 223, 160],
        Color.MAGENTA: [223, 160, 191],
        Color.RED: [255, 123, 96],
        Color.YELLOW: [230, 243, 136],
        Color.WHITE: [223, 223, 223],
        Color.BLACK: [185, 178, 158],
        Color.PURPLE: [187, 163, 226],
        Color.CYAN: [98, 239, 234],
    }

    # Colors used as cell edges
    cell_borders = {
        Color.LIGHT_BLUE: 'skyblue',
        Color.ORANGE: 'orange',
        Color.BLUE: 'blue',
        Color.GREEN: 'green',
        Color.MAGENTA: 'magenta',
        Color.RED: 'red',
        Color.YELLOW: 'yellow',
        Color.WHITE: 'lightgrey',
        Color.BLACK: 'darkgray',
        Color.PURPLE: 'purple',
        Color.CYAN: 'cyan'
    }

    # Load game image
    image = Image.open(image_path).convert('RGB')
    image_np = np.array(image)

    # Create a copy of the image to draw debug info on
    image_draw = draw = None
    if debug:
        image_draw = image.copy()
        draw = ImageDraw.Draw(image_draw)

    # Find the board bounding box
    board_box = find_boxes(image_np, [[0, 0, 0]], 150, False, 1250, None, None, 5, False)
    x_min, y_min, x_max, y_max = board_box[0]
    if debug: print(f"board box: {board_box[0]}")
    x_min, y_min, x_max, y_max = x_min + 13, y_min + 13, x_max - 13, y_max - 13

    # Find the board dimensions
    board_width, board_height = x_max - x_min, y_max - y_min
    if debug: print(f"board size: {board_width} x {board_height}")

    # Draw the border
    if debug: draw.rectangle(board_box[0], outline="grey", width=4)

    # Create a crop of just the board
    board_img_np = image_np[y_min:y_max, x_min:x_max]

    # Find starter cells. Only search within the board.
    starter_boxes = []
    for name, (target_colors, box_color, min_size, remove_nested, max_size, tolerance, merge_overlapping) in target_colors.items():
        boxes = find_boxes(board_img_np, target_colors, min_size, remove_nested, max_size, None, None, tolerance, merge_overlapping)
        boxes = [(x1 + x_min, y1 + y_min, x2 + x_min, y2 + y_min) for x1, y1, x2, y2 in boxes]
        draw_boxes(boxes, box_color, draw, width=1)
        starter_boxes.extend(boxes)

    # Find the width and height of the cells
    cell_width = max([box[2] - box[0] for box in starter_boxes])
    cell_height = max([box[3] - box[1] for box in starter_boxes])
    if debug: print(f"cell size: {cell_width:.1f} x {cell_height:.1f}")

    # Guess the number of rows and columns
    num_rows, num_cols = int(board_width / cell_width), int(board_height / cell_height)
    if debug: print(f"board dims: {num_rows} x {num_cols}")

    # Calculate the gap between each cell
    cell_gap = (board_width - (cell_width * num_cols)) / (num_cols - 1)
    if debug: print(f"cell gap: {cell_gap:.1f}")

    # Populate cells by sampling a pixel
    # TODO: Turn into a helper function, since this is used for both LinkedIn and the Tango App. If you do, careful with how pixel is calculated
    cells = [[None for _ in range(num_cols)] for _ in range(num_rows)]
    for r in range(num_rows):
        for c in range(num_cols):
            # Sample a pixel near the middle of the cell
            x = int(x_min + (cell_width + cell_gap) * c + 0.25 * cell_width)
            y = int(y_min + (cell_height + cell_gap) * r + 0.25 * cell_height)
            pixel = image_np[y, x, :]
            # Find which cell color this pixel is closest to
            distances = {name: np.linalg.norm(pixel - color) for name, color in cell_colors.items()}
            closest = min(distances, key=distances.get)
            cells[r][c] = closest
            # Draw the cell and type on the debug image
            if cell_borders[closest] is not None:
                x1 = x_min + (cell_width + cell_gap) * c
                y1 = y_min + (cell_height + cell_gap) * r
                x2, y2 = x1 + cell_width, y1 + cell_height
                rect = list(map(round, [x1, y1, x2, y2]))
                if debug: draw.rectangle(rect, outline=cell_borders[closest], width=6)

    # Show debug image
    if debug: image_draw.show()

    # Initialize puzzle
    board = Board([[Cell(i, j, color, Marker.BLANK) for j, color in enumerate(row)] for i, row in enumerate(cells)])
    
    return board

### Test Text Inputs

In [3]:
import time

# file_path = 'text_inputs/2025-05-04.txt'
# file_path = 'text_inputs/2025-05-05.txt'
# file_path = 'text_inputs/2025-05-07.txt'
# file_path = 'text_inputs/2025-05-08.txt'
file_path = 'text_inputs/2025-05-25-B.txt'
# file_path = 'text_inputs/2025-06-29-B.txt'
# file_path = 'text_inputs/2025-06-29.txt'
# file_path = 'text_inputs/2025-07-03.txt'
# file_path = 'text_inputs/2025-07-04.txt'

start = time.time()
board = Board.parse_from_text(file_path)
print(f"Parsed text in {time.time() - start:0.3f} seconds")

start = time.time()
solution = solve(board.copy())
print(f"Solved in {time.time() - start:0.3f} seconds")
print(board)
print(solution)

Parsed text in 0.001 seconds
Solved in 0.016 seconds
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                

*  *  *  *  *  *  *  *  Q  *  * 
*  *  *  *  *  Q  *  *  *  *  * 
*  Q  *  *  *  *  *  *  *  *  * 
*  *  *  *  *  *  Q  *  *  *  * 
*  *  *  Q  *  *  *  *  *  *  * 
*  *  *  *  *  *  *  Q  *  *  * 
*  *  *  *  *  *  *  *  *  *  Q 
*  *  *  *  Q  *  *  *  *  *  * 
*  *  *  *  *  *  *  *  *  Q  * 
Q  *  *  *  *  *  *  *  *  *  * 
*  *  Q  *  *  *  *  *  *  *  * 



### Test Image Inputs

In [4]:
# file_path = 'image_inputs/2025-05-04.png'
# file_path = 'image_inputs/2025-05-05.png'
# file_path = 'image_inputs/2025-05-07.png'
# file_path = 'image_inputs/2025-05-08.png'
# file_path = 'image_inputs/2025-05-25-B.png'
# file_path = 'image_inputs/2025-06-29-B.png'
# file_path = 'image_inputs/2025-06-29.png'
# file_path = 'image_inputs/2025-07-03.png'
# file_path = 'image_inputs/2025-07-04.png'
file_path = 'screenshots/2025-05-25-B-input.png'

start = time.time()
board = parse_linkedin_image(file_path, debug=False)
print(f"Parsed image in {time.time() - start:0.3f} seconds")

start = time.time()
solution = solve(board.copy())
print(f"Solved in {time.time() - start:0.3f} seconds")
print(board)
print(solution)

Parsed image in 0.906 seconds
Solved in 0.016 seconds
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                
                                

*  *  *  *  *  *  *  *  Q  *  * 
*  *  *  *  *  Q  *  *  *  *  * 
*  Q  *  *  *  *  *  *  *  *  * 
*  *  *  *  *  *  Q  *  *  *  * 
*  *  *  Q  *  *  *  *  *  *  * 
*  *  *  *  *  *  *  Q  *  *  * 
*  *  *  *  *  *  *  *  *  *  Q 
*  *  *  *  Q  *  *  *  *  *  * 
*  *  *  *  *  *  *  *  *  Q  * 
Q  *  *  *  *  *  *  *  *  *  * 
*  *  Q  *  *  *  *  *  *  *  * 

